# Libraries

In [1]:
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types


In [2]:
import pandas as pd
from collections import namedtuple
import requests as req
import io
import eventlet
import datetime
import requests

# Scrapper

In [ ]:
bashCommand = "curl https://18853.live.streamtheworld.com/BLURADIO_SC --output ./audio_scrapping/somess.mp3 --max-time 400"
os.system(bashCommand)

## Get the audio file

In [ ]:
import google.cloud.storage as storage

bashCommand = "curl https://18853.live.streamtheworld.com/BLURADIO_SC --output ./blue.mp3 --max-time 400"
os.system(bashCommand)
bashCommand = "ffmpeg -i ./blue.mp3.mp3 -c:v libx264 ./audio_scrapping/long.flac"
os.system(bashCommand)
bashCommand = "ffmpeg -i ./audio_scrapping/long.flac -ac 1 ./audio_scrapping/mono_long.flac"
os.system(bashCommand)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))
    
upload_blob("radioscrapping", "./audio_scrapping/mono_long.flac", "jupyter_tries/mono_long.flac")

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [3]:
from pydub import AudioSegment
song = AudioSegment.from_mp3("somess.mp3")

In [6]:
ss = song.export(format="flac", parameters=["-ac", "1"])

## Two commands to convert it 

In [4]:
bashCommand = "ffmpeg -i ./audio_scrapping/somess.mp3 -c:v libx264 ./audio_scrapping/long.flac"
os.system(bashCommand)

0

In [6]:
bashCommand = "ffmpeg -i ./audio_scrapping/long.flac -ac 1 ./audio_scrapping/mono_long.flac"
os.system(bashCommand)

0

In [12]:
bashCommand = "cp ./audio_scrapping/mono_long.flac gs://radioscrapping/mono_long.flac"
os.system(bashCommand)

256

## Upload to bucket 

### It has to read from a storage as the example is set like that

In [13]:
import google.cloud.storage as storage

In [14]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [15]:
upload_blob("radioscrapping", "./audio_scrapping/mono_long.flac", "jupyter_tries/mono_long.flac")

File ./audio_scrapping/mono_long.flac uploaded to jupyter_tries/mono_long.flac.


# Run the NLP speech to text

In [9]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        language_code='es-CO')

    operation = client.long_running_recognize(config, audio)

    print('Waiting for operation to complete...')
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    return response

# Results NLP 

In [16]:
ss = transcribe_gcs("gs://radioscrapping/jupyter_tries/mono_long.flac")

Waiting for operation to complete...


In [20]:
text1 = ss.results[0].alternatives[0].transcript

In [22]:
text2 = ss.results[1].alternatives[0].transcript

In [24]:
text3 =  ss.results[2].alternatives[0].transcript


In [25]:
tex4 = ss.results[3].alternatives[0].transcript


'en la transición las mujeres trans tienen todos los atributos atléticos de los hombres y ellos no permiten la participación de hombres en deportes femeninos porque en los niveles más altos las mujeres Sencillamente no pueden competir con los hombres entonces la transición hormonal a la que se someten las atletas tan les producen muchos cambios en sus cuerpos y condiciones atléticas por eso es muy importante que estos atletas hagan esos cambios antes de competir importante recordar que estamos conversando con Dios Ana Harper Ella es la coautora de las directrices y todo el reglamento para deportistas trans dentro del comité olímpico internacional tomando en cuenta la premisa de quién es usted señora Harvard para usted una mujer trans aún con el cambio sigue teniendo ventaja sobre la mujer en disciplinas por ejemplo como cuáles'

# NER

In [19]:
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import sys

def getsentimental(text):
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    return result.entities

### results

In [27]:
s = getsentimental(text1)

### The output is a class this is a function that makes it a json

In [34]:
def jsonit(result):
    listdict = []
    for entity in result:
        dictd = {}
        dictd["name"] = entity.name
        dictd["type"] = entity.type
        dictd["Salience"] = entity.salience
        mentionss = []
        for mention in entity.mentions:
            dict_m = {}
            dict_m["Content"] = mention.text.content
            dict_m["Magnitude"] = mention.sentiment.magnitude
            dict_m["Sentiment"] = mention.sentiment.score
            dict_m["Salience"] = entity.salience
            mentionss.append(dict_m)
        dictd["mentions"] = mentionss
        listdict.append(dictd)
    return listdict

In [56]:
sss = jsonit(s)

### loop to make it a dataframe

In [58]:
pre_df  = []
for mention in sss:
    Salienceg = mention["Salience"]
    types = mention["type"]
    name = mention["name"]
    men = mention['mentions']
    for mentis in men:
        Magnitude = mentis['Magnitude']
        Saliencei = mentis['Salience']
        Sentiment = mentis['Sentiment']
        tup = (Salienceg, types, name, Magnitude, Saliencei, Sentiment)
        pre_df.append(tup)

In [61]:
col = [ "Salienceg",
        "types", 
        "namei", 
        "Magnitude", 
        "Saliencei", 
        "Sentiment"
          ]
dfObj = pd.DataFrame(pre_df , columns=col)

In [62]:
dfObj

,Salienceg,types,namei,Magnitude,Saliencei,Sentiment
0,0.116637,1,coautora,0.1,0.116637,-0.1
1,0.084725,7,ámbito,0.1,0.084725,-0.1
2,0.079941,7,directrices,0.0,0.079941,0.0
3,0.079941,1,deportistas,0.0,0.079941,0.0
4,0.079941,3,comité,0.0,0.079941,0.0
5,0.074039,7,radio,0.1,0.074039,0.1
6,0.074039,7,conversación,0.0,0.074039,0.0
7,0.066863,7,reglamentos,0.0,0.066863,0.0
8,0.043613,1,directriz,0.0,0.043613,0.0
9,0.038460,7,expecting,0.0,0.038460,0.0


### Send that thing to big query

In [65]:
full_table_id = 'R_NER.Radio_tests'
project_id = 'proyecto-emiliano-isaza'
dfObj.to_gbq(full_table_id, project_id=project_id)

1it [00:02,  2.78s/it]


In [ ]:
from datetime import datetime
from google.cloud import bigquery
from pydub import AudioSegment
from io import BytesIO
from google.cloud import storage
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types